Ở đây mình sẽ dùng OpenRouter API để có thể access các LLM miễn phí

In [2]:
import os
from dotenv import load_dotenv
import requests, json

load_dotenv(".env", override=True)

True

In [3]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [38]:
import os
import json
import requests
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun


class CustomLLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom_llm"

    def _call(
        self,
        prompt: str,
        image_url: Optional[str] = None,  # Add optional image input
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
        }

        # Constructing messages with or without an image
        message_content = [{"type": "text", "text": prompt}]
        if image_url:
            message_content.append(
                {"type": "image_url", "image_url": {"url": image_url}}
            )

        data = {
            "model": "google/gemini-2.5-pro-exp-03-25:free",
            "messages": [
                {
                    "role": "user",
                    "content": message_content,
                }
            ],
        }

        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            data=json.dumps(data),
        )
        print(response.json())
        output = response.json()["choices"][0]["message"]["content"]

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [39]:
llm = CustomLLM(n=1)

In [ ]:
text_prompt = "What's in this image?"
image_url = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAIAQMAAAD+wSzIAAAABlBMVEX///+/v7+jQ3Y5AAAADklEQVQI12P4AIX8EAgALgAD/aNpbtEAAAAASUVORK5CYII"

response = llm._call(text_prompt, image_url=image_url)
print(response)

{'id': 'gen-1743263073-qOE1zPjclviER5j6qAms', 'provider': 'Google AI Studio', 'model': 'google/gemini-2.5-pro-exp-03-25', 'object': 'chat.completion', 'created': 1743263073, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': "This image shows a highly blurred abstract pattern.\n\nIt appears to be divided into four quadrants:\n\n*   **Top-left:** Blurry gray\n*   **Top-right:** Blurry white\n*   **Bottom-left:** Blurry white\n*   **Bottom-right:** Blurry gray\n\nThe lines dividing the quadrants are also very soft and blurred, creating smooth transitions between the areas. Due to the extreme blurriness, it's impossible to tell if this is a heavily out-of-focus photograph of something specific or a deliberately created abstract graphic.", 'refusal': None}}], 'usage': {'prompt_tokens': 270, 'completion_tokens': 118, 'total_tokens': 388}}
This image shows a highly blurred abstract pattern.

It appea